In [262]:
from utils.random_nfa_generator import generate
from utils.heuristics import *

import numpy as np

from copy import copy as copy2

from queue import PriorityQueue
from random import shuffle

import networkx as nx
from FAdo.conversions import *
from FAdo.reex import *

from utils.fadomata import *
from utils.reduction import *

In [263]:
n = np.random.randint(3, 7)  # np.random.randint(3, maxN)
k = 2  # np.random.choice([2, 5, 10])
d = np.random.choice([0.2])
gfa = generate(n, k, d, 'in-memory')


In [264]:
def is_epsilon(re: RegExp):
    #Converting re to str costs memory and time
    if type(re) == CEpsilon:
        return True
    else:
        return False

def in_included(re1: RegExp, re2: RegExp):
    """
    if re1 is included in re2, return 1
    if re1 is equivalent to re2, return 0
    if re2 is included in re1, return -1
    else return 2
    """
    #Can't catch the case when re1 = CDisj(CDisj(CAtom(1), CAtom(2)), CAtom(3)) and re2 = CDisj(CAtom(1), CDisj(CAtom(2), CAtom(3)))
    if re1 == re2:
        return 0
    #Trivial
    elif is_epsilon(re1) and is_epsilon(re2):
        return 0
    #Trivial
    elif is_epsilon(re1) and isinstance(re2, CStar):
        return 1
    #Trivial
    elif is_epsilon(re2) and isinstance(re1, CStar):
        return -1
    #re2 = arg1 + arg2 and re1 is arg1 or arg2
    elif isinstance(re2, CDisj) and (re2.arg1 == re1 or re2.arg2 == re1):
        return 1
    #Same as above
    elif isinstance(re1, CDisj) and (re1.arg1 == re2 or re1.arg2 == re2):
        return -1
    #re1.arg1 == re2.arg1 and re1.arg2 == re2.arg2 can never happen in this line so it all makes sense
    elif isinstance(re1, CDisj) and isinstance(re2, CDisj):
        if (in_included(re1.arg1, re2.arg2) == 0) and (in_included(re1.arg2, re2.arg1) == 0):
            return 0
    #This case will never happen since re1 == re2 can detect this case.
    #Virtually same as re1 == re2
    #might delete later
    elif isinstance(re1, CConcat) and isinstance(re2, CConcat):
        if (re1.arg1 == re2.arg1) and (re1.arg2 == re2.arg2):
            return 0
    #Trivial
    elif isinstance(re1, CStar) and isinstance(re2, CStar):
        return in_included(re1.arg, re2.arg)
    return 2


In [265]:
save_count_star = 0
save_count_concat = 0
save_count_disj = 0

all_count_star = 0
all_count_concat = 0
all_count_disj = 0


def eliminate_new(gfa: GFA, st: int):
    """Eliminate a state.

    :param int st: state to be eliminated"""        
    global save_count_concat, save_count_disj, save_count_star, all_count_concat, all_count_disj, all_count_star
    
    #Detect Self loop
    #r2: self loop string
    #epsilon* == epsilon and no duplicate star
    if st in gfa.delta and st in gfa.delta[st]:
        if isinstance(gfa.delta[st][st], reex.CStar) or is_epsilon(gfa.delta[st][st]):
            save_count_star += 1
            r2 = copy2(gfa.delta[st][st])
        else:
            r2 = copy2(reex.CStar(gfa.delta[st][st], copy2(gfa.Sigma)))
        del gfa.delta[st][st]
        all_count_star += 1
    else:
        r2 = None
    #r1: in-transtion string
    #r3: out-transition string
    for s in gfa.delta:
        if st not in gfa.delta[s]:
            continue
        r1 = copy2(gfa.delta[s][st])
        del gfa.delta[s][st]
        for s1 in gfa.delta[st]:
            r3 = copy2(gfa.delta[st][s1])
            if r2 is not None:
                if in_included(r2, r1) == 1 or in_included(r2, r3) == 1:
                    save_count_concat += 1
                    r = reex.CConcat(
                        r1, r3, copy2(gfa.Sigma))
                elif is_epsilon(r1):
                    save_count_concat += 1
                    if is_epsilon(r3):
                        r = r2
                    else:
                        r = reex.CConcat(r2, r3, copy2(gfa.Sigma))
                elif is_epsilon(r3):
                    save_count_concat += 1
                    r = reex.CConcat(r1, r2, copy2(gfa.Sigma))
                else:
                    r = reex.CConcat(r1, reex.CConcat(
                        r2, r3, copy2(gfa.Sigma)), copy2(gfa.Sigma))
            else:
                #there might be cases when r1 and r3 are all epsilon...?
                save_count_concat += 1
                if is_epsilon(r1):
                    r1 = None
                if is_epsilon(r3):
                    r3 = None
                if r1 == None and r3 == None:
                    r = CEpsilon()
                elif r1 == None:
                    r = r3
                elif r3 == None:
                    r = r1
                else:
                    save_count_concat -= 1
                    r = CConcat(r1, r3, copy2(gfa.Sigma))
            all_count_concat += 1

            #Merge w/ already existing label
            if s1 in gfa.delta[s]:
                print(f"R1: {r}, R2: {gfa.delta[s][s1]}")
                check_included = in_included(r, gfa.delta[s][s1])
                if check_included == 1 or check_included == 0:
                    save_count_disj += 1
                    gfa.delta[s][s1] = r
                elif check_included == -1:
                    save_count_disj += 1
                    gfa.delta[s][s1] = gfa.delta[s][s1]
                else:
                    if str(gfa.delta[s][s1]) > str(r):
                        gfa.delta[s][s1] = reex.CDisj(
                            r, gfa.delta[s][s1], copy2(gfa.Sigma))
                    else:
                        gfa.delta[s][s1] = reex.CDisj(
                            gfa.delta[s][s1], r, copy2(gfa.Sigma))
                all_count_disj += 1
            else:
                gfa.delta[s][s1] = r
    del gfa.delta[st]


def eliminate_by_state_weight_heuristic_new(gfa: GFA) -> RegExp:
    pq = PriorityQueue()
    for i in range(1, len(gfa.States) - 1):
        pq.put((get_weight(gfa, i), i))
    while not pq.empty():
        eliminate_new(gfa, pq.get()[1])
        # gfa.eliminate(pq.get()[1])
    if gfa.Initial in gfa.delta and gfa.Initial in gfa.delta[gfa.Initial]:
        return CConcat(CStar(gfa.delta[gfa.Initial][gfa.Initial]), gfa.delta[gfa.Initial][list(gfa.Final)[0]])
    else:
        return gfa.delta[gfa.Initial][list(gfa.Final)[0]]
    

def eliminate_by_repeated_state_weight_heuristic(gfa: GFA) -> RegExp:
    n = len(gfa.States) - 2
    victim = [i + 1 for i in range(len(gfa.States) - 2)]
    for i in range(n):
        if (len(victim) == 1):
            gfa.eliminate(victim[0])
            continue
        min_val = get_weight(gfa, victim[0])
        min_idx = 0
        for j in range(1, len(victim)):
            curr_val = get_weight(gfa, victim[j])
            if min_val > curr_val:
                min_val = curr_val
                min_idx = j
        gfa.eliminate(victim[min_idx])
        del victim[min_idx]
    if gfa.Initial in gfa.delta and gfa.Initial in gfa.delta[gfa.Initial]:
        return CConcat(CStar(gfa.delta[gfa.Initial][gfa.Initial]), gfa.delta[gfa.Initial][list(gfa.Final)[0]])
    else:
        return gfa.delta[gfa.Initial][list(gfa.Final)[0]]


def eliminate_by_repeated_state_weight_heuristic_new(gfa: GFA) -> RegExp:
    n = len(gfa.States) - 2
    victim = [i + 1 for i in range(len(gfa.States) - 2)]
    for i in range(n):
        if (len(victim) == 1):
            eliminate_new(gfa, victim[0])
            continue
        min_val = get_weight(gfa, victim[0])
        min_idx = 0
        for j in range(1, len(victim)):
            curr_val = get_weight(gfa, victim[j])
            if min_val > curr_val:
                min_val = curr_val
                min_idx = j
        eliminate_new(gfa, victim[min_idx])
        del victim[min_idx]
    if gfa.Initial in gfa.delta and gfa.Initial in gfa.delta[gfa.Initial]:
        return (CConcat(CStar(gfa.delta[gfa.Initial][gfa.Initial]), gfa.delta[gfa.Initial][list(gfa.Final)[0]]))
    else:
        return (gfa.delta[gfa.Initial][list(gfa.Final)[0]])


In [266]:
gfa_dup = gfa.dup()
print(len(str(eliminate_by_state_weight_heuristic(gfa_dup))))
gfa_dup = gfa.dup()
print(len(str(eliminate_by_state_weight_heuristic_new(gfa_dup))))
gfa_dup = gfa.dup()
print(len(str(eliminate_by_repeated_state_weight_heuristic(gfa_dup))))
gfa_dup = gfa.dup()
print(len(str(eliminate_by_repeated_state_weight_heuristic_new(gfa_dup))))
#gfa_dup = gfa.dup()
#print(len(str(eliminate_randomly(gfa_dup))))

print(f"Disj saved: {save_count_disj/all_count_disj} ({all_count_disj}), Concat saved: {save_count_concat/all_count_concat} ({all_count_concat}), Star saved: {save_count_star/all_count_star} ({all_count_star}).")

2277
R1: 1 (1* (0 + 1)), R2: 0
R1: 1 (1* 1), R2: 0
R1: 1 (1* 1), R2: 0
R1: 0 (1* (0 + 1)), R2: 0
R1: 0 (1* (0 + 1)), R2: 1
R1: 0 (1* 1), R2: 0 + 1
R1: (0 + (1 (1* 1))) ((0 + (1 (1* 1)))* 1), R2: 1
R1: (0 + (1 (1* 1))) ((0 + (1 (1* 1)))* 1), R2: 0 + 1
R1: (0 + (1 (1* 1))) (0 + (1 (1* 1)))*, R2: @epsilon
R1: (0 + (1 (1* 1))) ((0 + (1 (1* 1)))* (1 (1* (0 + 1)))), R2: 0 + (1 (1* (0 + 1)))
R1: (0 (1* 1)) ((0 + (1 (1* 1)))* 1), R2: 0 + 1
R1: (0 (1* 1)) ((0 + (1 (1* 1)))* 1), R2: 1
R1: (0 (1* 1)) (0 + (1 (1* 1)))*, R2: @epsilon
R1: (0 (1* 1)) ((0 + (1 (1* 1)))* (1 (1* (0 + 1)))), R2: 0 + (0 (1* (0 + 1)))
R1: ((0 (1* 1)) + (0 + 1)) ((0 + (1 (1* 1)))* 1), R2: 1
R1: ((0 (1* 1)) + (0 + 1)) ((0 + (1 (1* 1)))* 1), R2: 1
R1: ((0 (1* 1)) + (0 + 1)) ((0 + (1 (1* 1)))* (1 (1* (0 + 1)))), R2: (0 (1* (0 + 1))) + 1
R1: (((0 + (1 (1* 1))) ((0 + (1 (1* 1)))* (1 (1* (0 + 1))))) + (0 + (1 (1* (0 + 1))))) (((((0 (1* 1)) + (0 + 1)) ((0 + (1 (1* 1)))* (1 (1* (0 + 1))))) + ((0 (1* (0 + 1))) + 1))* ((((0 (1* 1)) +

In [30]:
gfa_dup = gfa.dup()
r1 = eliminate_by_repeated_state_weight_heuristic(gfa_dup)

In [161]:
gfa_dup = gfa.dup()
r2 = eliminate_by_state_weight_heuristic_new(gfa_dup)


In [162]:
str(r1)

'@epsilon (((((((3 + (((0 + 3) + 4) (4 (0* 2)))) + ((1 + (((0 + 3) + 4) ((0 + 2) + (4 (0* 2))))) (((3 (0* 2)) + (((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))))* (((((0 + 2) + 3) + (3 (0* 2))) + (((1 + 2) + 4) 4)) + (((2 + 3) + 4) (4 (0* 2))))))) + (((((0 + 3) + 4) 3) + ((1 + (((0 + 3) + 4) ((0 + 2) + (4 (0* 2))))) (((3 (0* 2)) + (((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))))* ((((1 + 2) + 4) ((0 + 1) + 3)) + (((2 + 3) + 4) 3))))) ((((4 + ((0 + 2) ((0 + 1) + 3))) + (2 3)) + (((2 (0* 2)) + (2 ((0 + 2) + (4 (0* 2))))) (((3 (0* 2)) + (((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))))* ((((1 + 2) + 4) ((0 + 1) + 3)) + (((2 + 3) + 4) 3)))))* ((((3 + (2 (0* 2))) + ((0 + 2) 4)) + (2 (4 (0* 2)))) + (((2 (0* 2)) + (2 ((0 + 2) + (4 (0* 2))))) (((3 (0* 2)) + (((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))))* (((((0 + 2) + 3) + (3 (0* 2))) + (((1 + 2) + 4) 4)) + (((2 + 3) + 4) (4 (0* 2)))))))))) + ((((((0 + 3) + 4) (4 + (4 (0* (0 + 4))))) + ((1 + (((0 + 3) + 4) ((0 + 2) + (4 (0* 2))))) (((3 (0* 2)) + (((2 + 3) + 4) ((0 + 2) + (4 (0* 

In [163]:
str(r2)

'((((((((((((((((0 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + 1) (((((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + (3 (0* 2)))* ((((2 + 3) + 4) (4 + (4 (0* (0 + 4))))) + ((3 (0* (0 + 4))) + 4)))) + (((0 + 3) + 4) (4 + (4 (0* (0 + 4)))))) (((((2 + 4) ((0 + 2) + (4 (0* 2)))) (((((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + (3 (0* 2)))* ((((2 + 3) + 4) (4 + (4 (0* (0 + 4))))) + ((3 (0* (0 + 4))) + 4)))) + ((2 + 4) (4 + (4 (0* (0 + 4))))))* ((((2 + 4) ((0 + 2) + (4 (0* 2)))) (((((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + (3 (0* 2)))* ((((2 + 3) + 4) (4 (0* 4))) + (2 + (3 (0* 4)))))) + (((2 + 4) (4 (0* 4))) + 3)))) + ((((((0 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + 1) (((((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + (3 (0* 2)))* ((((2 + 3) + 4) (4 (0* 4))) + (2 + (3 (0* 4)))))) + ((((0 + 3) + 4) (4 (0* 4))) + 1))) ((((((3 + (4 ((0 + 2) + (4 (0* 2))))) (((((2 + 3) + 4) ((0 + 2) + (4 (0* 2)))) + (3 (0* 2)))* ((((2 + 3) + 4) (4 + (4 (0* (0 + 4))))) + ((3 (0* (0 + 4))) + 4)))) + (1 + (4 (4 + (4 (0* (0 + 4))))))) (((((2 + 4) ((0 